# Aufgabenzuteilung

Open in Colab: [![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/febse/opt2025-de/blob/main/Job-Assignment.ipynb)

Eine Kreditratingagentur hat drei Mitarbeiter für ein Projekt zur Bewertung von strukturierten Finanzprodukten eingeteilt. Das Projekt hat drei wesentliche Unterprojekte: 

- Entwicklung von statistischen Modellen (Quantitativ)
- Recherche von Finanzprodukten, Literatur und Marktinformationen (Recherche)
- Entwicklung von qualitativen Modellen (Qualitativ)

Jeder Mitarbeiter muss ein Unterprojekt übernehmen. Die Mitarbeiter haben unterschiedliche Fähigkeiten und Präferenzen, die ihre Effizienz in den verschiedenen Unterprojekten beeinflussen. Die Effizienz der Mitarbeiter in den verschiedenen Unterprojekten ist wie folgt (größerer Wert bedeutet höhere Effizienz):

In [1]:
%pip install gurobipy

import pandas as pd
import gurobipy as gp
from gurobipy import GRB

dt = pd.DataFrame({
    'Quantitativ': [53, 27, 13],
    'Recherche': [80, 47, 67],
    'Qualitativ': [53, 73, 47]
}, index=['Boyko', 'Sasho', 'Radi'])
dt

/home/amarov/stats/opt2026-de/.venv/bin/python3: No module named pip


Note: you may need to restart the kernel to use updated packages.


,Quantitativ,Recherche,Qualitativ
Boyko,53,80,53
Sasho,27,47,73
Radi,13,67,47


Unsere Aufgabe ist es, die Mitarbeiter so zuzuweisen, dass die Gesamteffizienz maximiert wird.


## Das Modell

Jedes Modell besteht aus fünf Bestandteilen:

1. Mengen
2. Variablen
3. Parameter
4. Zielfunktion
5. Einschränkungen

In dieser Aufgabe haben wir zwei Mengen:

- Mitarbeiter: ($M = \{\text{Boyko}, \text{Sasho}, \text{Radi}\}$)
- Unterprojekte: ($P = \{\text{Quantitativ}, \text{Recherche}, \text{Qualitativ}\}$)

Die Entscheidungsvariablen sind:

$x_{ij} \in \{0, 1\}$: 1, wenn Mitarbeiter $i$ Unterprojekt $j$ übernimmt, 0 sonst (also binäre Variablen)

Die Parameter sind hier die Effizienz der Mitarbeiter in den verschiedenen Unterprojekten:

$e_{ij}$: Effizienz von Mitarbeiter $i$ in Unterprojekt $j$

Die Zielfunktion ist die Gesamteffizienz:

$\max \sum_{i \in M} \sum_{j \in P} e_{ij} \cdot x_{ij}$

Das ist eine Kurzform für:

$$
\begin{align*}
\max & \quad e_{\text{Boyko}, \text{Quantitativ}} \cdot x_{\text{Boyko}, \text{Quantitativ}} + e_{\text{Boyko}, \text{Recherche}} \cdot x_{\text{Boyko}, \text{Recherche}} + e_{\text{Boyko}, \text{Qualitativ}} \cdot x_{\text{Boyko}, \text{Qualitativ}} \\
& + e_{\text{Sasho}, \text{Quantitativ}} \cdot x_{\text{Sasho}, \text{Quantitativ}} + e_{\text{Sasho}, \text{Recherche}} \cdot x_{\text{Sasho}, \text{Recherche}} + e_{\text{Sasho}, \text{Qualitativ}} \cdot x_{\text{Sasho}, \text{Qualitativ}} \\
& + e_{\text{Radi}, \text{Quantitativ}} \cdot x_{\text{Radi}, \text{Quantitativ}} + e_{\text{Radi}, \text{Recherche}} \cdot x_{\text{Radi}, \text{Recherche}} + e_{\text{Radi}, \text{Qualitativ}} \cdot x_{\text{Radi}, \text{Qualitativ}}
\end{align*}
$$

Die Einschränkungen sind:

1. Jeder Mitarbeiter muss genau ein Unterprojekt übernehmen:

$\sum_{j \in P} x_{ij} = 1 \quad \forall i \in M$

Dies ist eine Kurzform für:

$$
\begin{align*}
i = \text{Boyko} & \quad \Rightarrow \quad x_{\text{Boyko}, \text{Quantitativ}} + x_{\text{Boyko}, \text{Recherche}} + x_{\text{Boyko}, \text{Qualitativ}} = 1 \\
i = \text{Sasho} & \quad \Rightarrow \quad x_{\text{Sasho}, \text{Quantitativ}} + x_{\text{Sasho}, \text{Recherche}} + x_{\text{Sasho}, \text{Qualitativ}} = 1 \\
i = \text{Radi} & \quad \Rightarrow \quad x_{\text{Radi}, \text{Quantitativ}} + x_{\text{Radi}, \text{Recherche}} + x_{\text{Radi}, \text{Qualitativ}} = 1 \\
\end{align*}
$$

2. Jedes Unterprojekt muss von genau einem Mitarbeiter übernommen werden:

$\sum_{i \in M} x_{ij} = 1 \quad \forall j \in P$

Dies ist eine Kurzform für:

$$
\begin{align*}
j = \text{Quantitativ} & \quad \Rightarrow \quad x_{\text{Boyko}, \text{Quantitativ}} + x_{\text{Sasho}, \text{Quantitativ}} + x_{\text{Radi}, \text{Quantitativ}} = 1 \\
j = \text{Recherche} & \quad \Rightarrow \quad x_{\text{Boyko}, \text{Recherche}} + x_{\text{Sasho}, \text{Recherche}} + x_{\text{Radi}, \text{Recherche}} = 1 \\
j = \text{Qualitativ} & \quad \Rightarrow \quad x_{\text{Boyko}, \text{Qualitativ}} + x_{\text{Sasho}, \text{Qualitativ}} + x_{\text{Radi}, \text{Qualitativ}} = 1 \\
\end{align*}
$$

## Umsetzung


In [2]:

m = gp.Model('Aufgabenzuteilung')

# Die Variablen x_{ij} geben an, ob Aufgabe i an Person j zugewiesen wird

x = m.addVars(dt.index, dt.columns, vtype=GRB.BINARY, name='assign')

# Die Zielfunktion ist die Summe der Produkte der Qualifikationen und der Zuweisungen

m.setObjective(gp.quicksum(dt.loc[i, j] * x[i, j] for i in dt.index for j in dt.columns), GRB.MAXIMIZE)

# Jeder Mitarbeiter kann nur eine Aufgabe erhalten

m.addConstrs((x.sum(i, '*') == 1 for i in dt.index),'Mitarbeiter')

# Jede Aufgabe kann nur einem Mitarbeiter zugewiesen werden

m.addConstrs((x.sum('*', j) == 1 for j in dt.columns), 'Aufgabe')

m.optimize()

# Die Lösung as pandas DataFrame

solution = pd.DataFrame(
    ((i, j, x[i, j].x) for i in dt.index for j in dt.columns),
    columns=['Mitarbeiter', 'Aufgabe', 'Zuweisung']
)
solution

Restricted license - for non-production use only - expires 2027-11-29


Gurobi Optimizer version 13.0.1 build v13.0.1rc0 (linux64 - "Ubuntu 24.04.4 LTS")


CPU model: Intel(R) Core(TM) i9-14900K, instruction set [SSE2|AVX|AVX2]


Thread count: 32 physical cores, 32 logical processors, using up to 32 threads


Optimize a model with 6 rows, 9 columns and 18 nonzeros (Max)


...

Thread count was 32 (of 32 available processors)


Solution count 3: 193 147 113 


Optimal solution found (tolerance 1.00e-04)


Best objective 1.930000000000e+02, best bound 1.930000000000e+02, gap 0.0000%


,Mitarbeiter,Aufgabe,Zuweisung
0,Boyko,Quantitativ,1.0
1,Boyko,Recherche,-0.0
2,Boyko,Qualitativ,-0.0
3,Sasho,Quantitativ,-0.0
4,Sasho,Recherche,-0.0
5,Sasho,Qualitativ,1.0
6,Radi,Quantitativ,-0.0
7,Radi,Recherche,1.0
8,Radi,Qualitativ,-0.0


## Wie funktioniert der Code?

Hier haben wir die Funktion `gp.quicksum()` verwendet, um die Summe in der Zielfunktion zu bilden. Weiterhin haben wir die Methode `x.sum()` eingeführt, um die Summen für die Einschränkungen zu bilden. 

In [3]:
# Mit dt.loc кönnen wir auf einzelne Werte in dem DataFrame zugreifen
# zum Beispiel

dt.loc['Boyko', 'Quantitativ']  # Zugriff auf den Wert in der Zeile 'Boyko' und Spalte 'Quantitativ'

np.int64(53)

In [4]:
# Die doppelte Schleife iteriert über alle Zeilen und alle Spalten im DataFrame

list((dt.loc[i, j] for i in dt.index for j in dt.columns))

[np.int64(53),
 np.int64(80),
 np.int64(53),
 np.int64(27),
 np.int64(47),
 np.int64(73),
 np.int64(13),
 np.int64(67),
 np.int64(47)]

In [5]:
list((x[i, j] for i in dt.index for j in dt.columns))

[<gurobi.Var assign[Boyko,Quantitativ] (value 1.0)>,
 <gurobi.Var assign[Boyko,Recherche] (value -0.0)>,
 <gurobi.Var assign[Boyko,Qualitativ] (value -0.0)>,
 <gurobi.Var assign[Sasho,Quantitativ] (value -0.0)>,
 <gurobi.Var assign[Sasho,Recherche] (value -0.0)>,
 <gurobi.Var assign[Sasho,Qualitativ] (value 1.0)>,
 <gurobi.Var assign[Radi,Quantitativ] (value -0.0)>,
 <gurobi.Var assign[Radi,Recherche] (value 1.0)>,
 <gurobi.Var assign[Radi,Qualitativ] (value -0.0)>]

In [6]:
gp.quicksum(dt.loc[i, j] * x[i, j] for i in dt.index for j in dt.columns)

<gurobi.LinExpr: 53.0 assign[Boyko,Quantitativ] + 80.0 assign[Boyko,Recherche] + 53.0 assign[Boyko,Qualitativ] + 27.0 assign[Sasho,Quantitativ] + 47.0 assign[Sasho,Recherche] + 73.0 assign[Sasho,Qualitativ] + 13.0 assign[Radi,Quantitativ] + 67.0 assign[Radi,Recherche] + 47.0 assign[Radi,Qualitativ]>

In [7]:
x.sum('*', 'Recherche') == 1

<gurobi.TempConstr: assign[Boyko,Recherche] + assign[Sasho,Recherche] + assign[Radi,Recherche] == 1>

In [8]:
list((x.sum('*', j) for j in dt.columns))

[<gurobi.LinExpr: assign[Boyko,Quantitativ] + assign[Sasho,Quantitativ] + assign[Radi,Quantitativ]>,
 <gurobi.LinExpr: assign[Boyko,Recherche] + assign[Sasho,Recherche] + assign[Radi,Recherche]>,
 <gurobi.LinExpr: assign[Boyko,Qualitativ] + assign[Sasho,Qualitativ] + assign[Radi,Qualitativ]>]

In [9]:
x.sum('Boyko', '*') == 1

<gurobi.TempConstr: assign[Boyko,Quantitativ] + assign[Boyko,Recherche] + assign[Boyko,Qualitativ] == 1>

In [10]:
list((x.sum(i, '*') for i in dt.index))

[<gurobi.LinExpr: assign[Boyko,Quantitativ] + assign[Boyko,Recherche] + assign[Boyko,Qualitativ]>,
 <gurobi.LinExpr: assign[Sasho,Quantitativ] + assign[Sasho,Recherche] + assign[Sasho,Qualitativ]>,
 <gurobi.LinExpr: assign[Radi,Quantitativ] + assign[Radi,Recherche] + assign[Radi,Qualitativ]>]

In [11]:
# Um die Zuweisungen zu visualisieren, können wir die Tabelle umformen

solution.pivot(index='Mitarbeiter', columns='Aufgabe', values='Zuweisung')

Aufgabe,Qualitativ,Quantitativ,Recherche
Mitarbeiter,,,
Boyko,-0.0,1.0,-0.0
Radi,-0.0,-0.0,1.0
Sasho,1.0,-0.0,-0.0


In [12]:
m.write('Aufgabenzuteilung.lp')

with open('Aufgabenzuteilung.lp', 'r') as f:
    print(f.read())

\ Model Aufgabenzuteilung
\ LP format - for model browsing. Use MPS format to capture full model detail.
Maximize
  53 assign[Boyko,Quantitativ] + 80 assign[Boyko,Recherche]
   + 53 assign[Boyko,Qualitativ] + 27 assign[Sasho,Quantitativ]
   + 47 assign[Sasho,Recherche] + 73 assign[Sasho,Qualitativ]
   + 13 assign[Radi,Quantitativ] + 67 assign[Radi,Recherche]
   + 47 assign[Radi,Qualitativ]
Subject To
 Mitarbeiter[Boyko]: assign[Boyko,Quantitativ] + assign[Boyko,Recherche]
   + assign[Boyko,Qualitativ] = 1
 Mitarbeiter[Sasho]: assign[Sasho,Quantitativ] + assign[Sasho,Recherche]
   + assign[Sasho,Qualitativ] = 1
 Mitarbeiter[Radi]: assign[Radi,Quantitativ] + assign[Radi,Recherche]
   + assign[Radi,Qualitativ] = 1
 Aufgabe[Quantitativ]: assign[Boyko,Quantitativ]
   + assign[Sasho,Quantitativ] + assign[Radi,Quantitativ] = 1
 Aufgabe[Recherche]: assign[Boyko,Recherche] + assign[Sasho,Recherche]
   + assign[Radi,Recherche] = 1
 Aufgabe[Qualitativ]: assign[Boyko,Qualitativ] + assign[Sasho,Qua

Einen alternativen Ansatz zur Modellierung wäre es, zuerst den Datensatz umzustrukturieren, so dass die Effizienz der Mitarbeiter in den verschiedenen Unterprojekten in einer Spalte steht (Langform). Dann könnten wir `x.prod()` verwenden und die Schleife in der Zielfunktion vermeiden.

In [13]:
# Den Datensatz in Langform bringen. Das Ergebnis ist eine Serie mit einem MultiIndex

dt_long = dt.stack()
dt_long

Boyko  Quantitativ    53
       Recherche      80
       Qualitativ     53
Sasho  Quantitativ    27
       Recherche      47
       Qualitativ     73
Radi   Quantitativ    13
       Recherche      67
       Qualitativ     47
dtype: int64

In [14]:
dt_long.index

MultiIndex([('Boyko', 'Quantitativ'),
            ('Boyko',   'Recherche'),
            ('Boyko',  'Qualitativ'),
            ('Sasho', 'Quantitativ'),
            ('Sasho',   'Recherche'),
            ('Sasho',  'Qualitativ'),
            ( 'Radi', 'Quantitativ'),
            ( 'Radi',   'Recherche'),
            ( 'Radi',  'Qualitativ')],
           )

In [15]:
# Auf die einzelnen Werte des MultiIndex zugreifen

dt_long.index.levels[0]

Index(['Boyko', 'Sasho', 'Radi'], dtype='str')

In [16]:
dt_long.index.levels[1]

Index(['Quantitativ', 'Recherche', 'Qualitativ'], dtype='str')

In [17]:
# Die Serie in ein Dictionary umwandeln

dt_long.to_dict()

{('Boyko', 'Quantitativ'): 53,
 ('Boyko', 'Recherche'): 80,
 ('Boyko', 'Qualitativ'): 53,
 ('Sasho', 'Quantitativ'): 27,
 ('Sasho', 'Recherche'): 47,
 ('Sasho', 'Qualitativ'): 73,
 ('Radi', 'Quantitativ'): 13,
 ('Radi', 'Recherche'): 67,
 ('Radi', 'Qualitativ'): 47}

In [18]:
list((x.sum(i, '*') == 1 for i in dt_long.index.levels[0]))

[<gurobi.TempConstr: assign[Boyko,Quantitativ] + assign[Boyko,Recherche] + assign[Boyko,Qualitativ] == 1>,
 <gurobi.TempConstr: assign[Sasho,Quantitativ] + assign[Sasho,Recherche] + assign[Sasho,Qualitativ] == 1>,
 <gurobi.TempConstr: assign[Radi,Quantitativ] + assign[Radi,Recherche] + assign[Radi,Qualitativ] == 1>]

In [19]:
m1 = gp.Model('Aufgabenzuteilung 1')
m1.Params.LogToConsole = 0

x = m1.addVars(dt_long.index, vtype=GRB.BINARY, name='assign')

m1.setObjective(x.prod(dt_long.to_dict()), GRB.MAXIMIZE)

m1.addConstrs((x.sum(mitarbeiter, '*') == 1 for mitarbeiter in dt_long.index.levels[0]), 'Mitarbeiter')
m1.addConstrs((x.sum('*', unterprojekt) == 1 for unterprojekt in dt_long.index.levels[1]), 'Unterprojekt')

m1.write('Aufgabenzuteilung 1.lp')

with open('Aufgabenzuteilung 1.lp', 'r') as f:
    print(f.read())

m1.optimize()

solution1 = pd.DataFrame(
    ((mitarbeiter, unterprojekt, x[mitarbeiter, unterprojekt].x) for mitarbeiter, unterprojekt in dt_long.index),
    columns=['Mitarbeiter', 'Unterprojekt', 'Zuweisung']
)

solution1.pivot(index='Mitarbeiter', columns='Unterprojekt', values='Zuweisung')

Set parameter LogToConsole to value 0


\ Model Aufgabenzuteilung 1
\ LP format - for model browsing. Use MPS format to capture full model detail.
Maximize
  53 assign[Boyko,Quantitativ] + 80 assign[Boyko,Recherche]
   + 53 assign[Boyko,Qualitativ] + 27 assign[Sasho,Quantitativ]
   + 47 assign[Sasho,Recherche] + 73 assign[Sasho,Qualitativ]
   + 13 assign[Radi,Quantitativ] + 67 assign[Radi,Recherche]
   + 47 assign[Radi,Qualitativ]
Subject To
 Mitarbeiter[Boyko]: assign[Boyko,Quantitativ] + assign[Boyko,Recherche]
   + assign[Boyko,Qualitativ] = 1
 Mitarbeiter[Sasho]: assign[Sasho,Quantitativ] + assign[Sasho,Recherche]
   + assign[Sasho,Qualitativ] = 1
 Mitarbeiter[Radi]: assign[Radi,Quantitativ] + assign[Radi,Recherche]
   + assign[Radi,Qualitativ] = 1
 Unterprojekt[Quantitativ]: assign[Boyko,Quantitativ]
   + assign[Sasho,Quantitativ] + assign[Radi,Quantitativ] = 1
 Unterprojekt[Recherche]: assign[Boyko,Recherche] + assign[Sasho,Recherche]
   + assign[Radi,Recherche] = 1
 Unterprojekt[Qualitativ]: assign[Boyko,Qualitativ]
 

Unterprojekt,Qualitativ,Quantitativ,Recherche
Mitarbeiter,,,
Boyko,-0.0,1.0,-0.0
Radi,-0.0,-0.0,1.0
Sasho,1.0,-0.0,-0.0
